<a href="https://colab.research.google.com/github/jorge-flores-py/PROCESAMIENTO-DE-HABLA-IAPH/blob/main/TP4_chatbot_retrieval_JORGE_FLORES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install spacy --quiet
!python -m spacy download es_core_news_sm --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 94.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load("es_core_news_sm")
import es_core_news_sm
nlp = es_core_news_sm.load()
doc = nlp("Esto es una frase.")
print([(w.text, w.pos_) for w in doc])

# Chatbots basados en recuperación

En inglés information retrieval chatbots

# Motor de búsqueda

* Búsqueda por palabras clave: Extrae palabras clave de la pregunta del usuario y busca coincidencias en las preguntas almacenadas.

* Similitud del coseno: Si has representado las preguntas como vectores (por ejemplo, usando TF-IDF o word embeddings), puedes usar la similitud del coseno para medir la distancia entre las preguntas.

* Word embeddings: Utiliza modelos de word embeddings como Word2Vec o BERT para obtener representaciones semánticas de las preguntas y las consultas del usuario.

### Búsqueda por palabras claves

In [ ]:
tu_diccionario = {
   "hola": "¡Hola! ¿En qué puedo ayudarte?",
   "adiós": "Hasta luego. ¡Que tengas un buen día!",
   "información": "¿Qué tipo de información estás buscando?",
   # Agrega más entradas de diccionario según tus necesidades
}


In [ ]:
def responder_pregunta(pregunta):
    pregunta_procesada = nlp(pregunta.lower())  # Procesa la pregunta y convierte a minúsculas
    respuesta = "Lo siento, no entiendo tu pregunta."

    # Busca una coincidencia en el diccionario
    for palabra in pregunta_procesada:
        # regresa la primer coincidencia que encuentra
        if palabra.text in tu_diccionario:
            respuesta = tu_diccionario[palabra.text]
            break

    return respuesta


In [ ]:
while True:
    entrada_usuario = input("Tú: ")
    if entrada_usuario.lower() == "salir":
        print("Chatbot: Hasta luego.")
        break
    respuesta = responder_pregunta(entrada_usuario)
    print("Chatbot:", respuesta)


## Búsqueda por similitud

Para los chatbots basados ​​en recuperación, es común utilizar bolsas de palabras (bag of words) o tf-idf para calcular la similitud de intenciones.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Datos de ejemplo
preguntas = ["¿Qué es el aprendizaje automático?",
             "¿Cómo funciona la regresión lineal?"]
respuestas = ["El aprendizaje automático es una rama de la inteligencia artificial...",
              "La regresión lineal es un método de modelado..."]

# Vectorización con TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preguntas)

# Función para encontrar la mejor coincidencia
def responder_pregunta(consulta_usuario):
    consulta_vec = vectorizer.transform([consulta_usuario])
    similitudes = cosine_similarity(consulta_vec, tfidf_matrix).flatten()
    print(similitudes)
    indice_mejor_coincidencia = similitudes.argmax()
    print(indice_mejor_coincidencia)
    return respuestas[indice_mejor_coincidencia]


In [ ]:

# Ejemplo de consulta
consulta = "¿Qué es la regresión lineal?"
print(responder_pregunta(consulta))


## Búsqueda por similitud en embeddings

Puedes vectorizar el texto usando embeddings, como vimos la clase pasada.


## Actividades

### 1) Elaborar un dataset de preguntas y respuestas para crear un Chatbot para un aplicación particular. ( 3 puntos )

1.1 Debe definir la aplicación (atención al cliente bancario, atención a estudiantes universitarios, etc).
1.2 El listado de preguntas y respuestas debe tener como mínimo 20 elementos pregunta - respuesta.

###  2) Crear el chatbot utilizando TFIDF y similitud del coseno. (1 punto)

### 3) Crear otro chatbot utilizando embeddings. Indique cuál embedding (1 punto) pre-entrenado eligió.

### 4) Muestra ambos chatbots funcionando (1 punto)

Adjuntar la lista de preguntas utilizadas para probar el funcionamiento.

### 5) Añade tus conclusiones de todo lo realizado (2 punto)

### 6) BONUS: usa lo realizado en 1 y 3 para crear un chatbot RAG. (2 puntos)

* Utiliza un modelo LLM pre-entrenado.

* Este punto no es obligatorio de realizar para quienes quieran regularizar / recuperar y luego rendirán en mesa.
* Para quienes tienen condiciones para promocionar (han realizado y entregado los TPs a tiempo) la resolución de este ejercicio será tenida en cuenta para sumar a la promoción.

### 7) No olvides:

* Explicar tus decisiones y configuraciones. Añadir tus conclusiones.
* Anunciar en el foro cuál será tu aplicación y postear tu entrega y tus avances.
* Debes subir tu notebook a un repo GitHub público de tu propiedad compartido + enlace colab.
* Documentar todo el proceso.





## 1) Elaborar un dataset de preguntas y respuestas para crear un Chatbot para un aplicación particular. ( 3 puntos )

In [ ]:
!pip install -U sentence-transformers


In [29]:
import re
import spacy
import pandas as pd
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
nlp = spacy.load("es_core_news_sm")
stopwords_es = set(STOP_WORDS)


In [31]:
def limpiar_texto(text):
    text = text.lower()
    text = re.sub(r'[.!¿?¡,"\':“”«»…]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^a-zñáéíóúü\s]', '', text)
    return text

def lematizar_y_filtrar(text):
    doc = nlp(text)
    lemas = [token.lemma_ for token in doc if token.lemma_ not in stopwords_es and token.is_alpha]
    return " ".join(lemas)

def procesar_preguntas(preguntas):
    preguntas_limpias = [limpiar_texto(p) for p in preguntas]
    return [lematizar_y_filtrar(p) for p in preguntas_limpias]

In [7]:
faq_data = [
    ("¿Cómo me inscribo a una materia?", "Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'."),
    ("¿Dónde veo mis notas?", "Podés ver tus notas desde el menú 'Historial Académico' en el sistema de autogestión."),
    ("¿Qué pasa si me llevo una materia?", "Deberás recursarla el próximo cuatrimestre."),
    ("¿Puedo cambiarme de turno?", "Sí, durante el período de ajuste podés cambiarte de turno desde el sistema."),
    ("¿Cómo consulto mis faltas?", "Las faltas se consultan desde la sección 'Asistencia' del sistema de alumnos."),
    ("¿Dónde pido el certificado de alumno regular?", "Desde 'Trámites Online' podés solicitarlo y descargarlo."),
    ("¿Cuántas materias puedo cursar por cuatrimestre?", "Podés cursar hasta 5 materias por cuatrimestre."),
    ("¿Qué es el SIU Guaraní?", "Es el sistema de gestión académica donde gestionás tus materias y notas."),
    ("¿Cómo consulto el calendario académico?", "El calendario está disponible en la web oficial del instituto."),
    ("¿Dónde veo mi promedio?", "Tu promedio está disponible en el 'Historial Académico' dentro del sistema."),
    ("¿Qué hago si tengo problemas con el campus virtual?", "Contactá a soporte técnico o enviá un mail a soporte@universidad.edu."),
    ("¿Puedo recursar una materia aprobada?", "No, sólo podés recursar materias desaprobadas."),
    ("¿Qué pasa si abandono una materia?", "Figurarás como ausente y no afectará tu promedio."),
    ("¿Cómo pido una equivalencia?", "Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios."),
    ("¿Cuándo son las inscripciones?", "Las inscripciones se habilitan una semana antes del inicio del cuatrimestre."),
    ("¿Qué es una correlativa?", "Es una materia que debés aprobar antes de cursar otra."),
    ("¿Cómo consulto mi situación académica?", "Desde 'Historial Académico' podés ver tu situación actual."),
    ("¿Dónde encuentro el material de estudio?", "Todo el material está en el campus virtual en cada materia."),
    ("¿Puedo rendir un final libre?", "Sí, pero tenés que anotarte en los turnos establecidos."),
    ("¿Cómo me contacto con un profesor?", "Podés enviarle un mensaje desde el campus o escribirle a su mail institucional.")
]


In [11]:
preguntas_raw = [p for p, r in faq_data]
respuestas = [r for p, r in faq_data]

preguntas_procesadas = procesar_preguntas(preguntas_raw)

##2) Crear el chatbot utilizando TFIDF y similitud del coseno. (1 punto)

In [32]:
class ChatbotTFIDF:
    # Método que se ejecuta al crear el chatbot
    def __init__(self, preguntas, respuestas):
        self.vectorizer = TfidfVectorizer()  # Crea el vectorizador TF-IDF
        self.X = self.vectorizer.fit_transform(preguntas)  # Calcula los vectores TF-IDF de las preguntas
        self.respuestas = respuestas  # Guarda la lista de respuestas posibles

    # Método para responder una pregunta del usuario
    def responder(self, pregunta_usuario):
        pregunta_proc = procesar_preguntas([pregunta_usuario])[0]  # Limpia y lematiza la pregunta del usuario
        pregunta_vec = self.vectorizer.transform([pregunta_proc])  # Convierte la pregunta a vector TF-IDF
        similitudes = cosine_similarity(pregunta_vec, self.X)  # Calcula la similitud entre la pregunta y las preguntas del FAQ
        idx = similitudes.argmax()  # Toma el índice de la pregunta más parecida
        return self.respuestas[idx]  # Devuelve la respuesta correspondiente



In [23]:
# Ejemplo de uso
chatbot_tfidf = ChatbotTFIDF(preguntas_procesadas, respuestas)
#print(chatbot_tfidf.responder("¿Dónde veo mis calificaciones?"))
print(chatbot_tfidf.responder("como me inscribo?"))


Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.


##3) Crear otro chatbot utilizando embeddings. Indique cuál embedding (1 punto) pre-entrenado eligió.

In [24]:
from sentence_transformers import SentenceTransformer
import numpy as np

class ChatbotEmbeddings:
    # Constructor del chatbot, recibe los datos (pregunta, respuesta)
    def __init__(self, data):
        self.modelo = SentenceTransformer('all-MiniLM-L6-v2')  # Carga el modelo de embeddings preentrenado
        self.preguntas = [item[0] for item in data]  # Extrae todas las preguntas del conjunto de datos
        self.respuestas = [item[1] for item in data]  # Extrae todas las respuestas correspondientes
        self.embeddings = self.modelo.encode(self.preguntas, convert_to_numpy=True)  # Calcula los embeddings de las preguntas

    # Método para responder una pregunta del usuario
    def responder(self, pregunta_usuario):
        emb_usuario = self.modelo.encode([pregunta_usuario], convert_to_numpy=True)[0]  # Genera el embedding de la pregunta del usuario
        # Calcula la similitud del coseno entre la pregunta del usuario y cada pregunta del dataset
        similitudes = np.dot(self.embeddings, emb_usuario) / (
            np.linalg.norm(self.embeddings, axis=1) * np.linalg.norm(emb_usuario))
        idx = np.argmax(similitudes)  # Obtiene el índice de la pregunta más parecida (mayor similitud)
        return self.respuestas[idx]  # Devuelve la respuesta correspondiente



In [34]:
# Ejemplo de uso
chatbot_emb = ChatbotEmbeddings(faq_data)
print(chatbot_emb.responder("situación académica"))


Desde 'Historial Académico' podés ver tu situación actual.


##4) Muestra ambos chatbots funcionando (1 punto)

In [26]:
preguntas_test = [
    "cómo hago para anotarme en una materia",
    "dónde puedo revisar mis calificaciones",
    "cuántas materias se pueden hacer por cuatrimestre",
    "qué pasa si abandono una materia",
    "me quiero cambiar de turno, ¿puedo?",
    "dónde puedo ver mi promedio",
    "cómo se piden las equivalencias",
    "me falló el campus virtual, no entra",
    "cuándo arrancan las inscripciones",
    "cómo me comunico con un docente"
]


def comparar_bots_df(preguntas_usuario, bot_tfidf, bot_emb):
    resultados = []
    for pregunta in preguntas_usuario:
        resp_tfidf = bot_tfidf.responder(pregunta)
        resp_emb = bot_emb.responder(pregunta)
        resultados.append({
            "Pregunta": pregunta,
            "Respuesta TF-IDF": resp_tfidf,
            "Respuesta Embeddings": resp_emb
        })

    df_resultados = pd.DataFrame(resultados)
    pd.set_option('display.max_colwidth', None)  # <-- mostrar columnas completas
    return df_resultados


In [27]:
df_comparacion = comparar_bots_df(preguntas_test,chatbot_tfidf,chatbot_emb)
display(df_comparacion)

,Pregunta,Respuesta TF-IDF,Respuesta Embeddings
0,cómo hago para anotarme en una materia,"Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.","Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'."
1,dónde puedo revisar mis calificaciones,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.",Podés ver tus notas desde el menú 'Historial Académico' en el sistema de autogestión.
2,cuántas materias se pueden hacer por cuatrimestre,Podés cursar hasta 5 materias por cuatrimestre.,Podés cursar hasta 5 materias por cuatrimestre.
3,qué pasa si abandono una materia,Figurarás como ausente y no afectará tu promedio.,Figurarás como ausente y no afectará tu promedio.
4,"me quiero cambiar de turno, ¿puedo?","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema."
5,dónde puedo ver mi promedio,Tu promedio está disponible en el 'Historial Académico' dentro del sistema.,Tu promedio está disponible en el 'Historial Académico' dentro del sistema.
6,cómo se piden las equivalencias,Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios.,Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios.
7,"me falló el campus virtual, no entra",Contactá a soporte técnico o enviá un mail a soporte@universidad.edu.,Contactá a soporte técnico o enviá un mail a soporte@universidad.edu.
8,cuándo arrancan las inscripciones,Las inscripciones se habilitan una semana antes del inicio del cuatrimestre.,Las inscripciones se habilitan una semana antes del inicio del cuatrimestre.
9,cómo me comunico con un docente,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.",Podés enviarle un mensaje desde el campus o escribirle a su mail institucional.


##5) Añade tus conclusiones de todo lo realizado (2 punto)



### Conclusión

El chatbot basado en embeddings ofrece mejores resultados que el de TF-IDF cuando la redacción de las preguntas cambia, ya que entiende mejor el significado general. En cambio, TF-IDF funciona bien solo si la pregunta del usuario se parece mucho a la original. Por eso, embeddings es más útil para situaciones reales donde las personas no escriben siempre de forma exacta.

---



### Fuentes

- Apuntes de clase proporcionados por el docente de la materia.
- Asistencia de ChatGPT (modelo de lenguaje de OpenAI) para comprensión de código, explicaciones teóricas y comentarios en Python.
